In [ ]:
import pandas as pd
fields = pd.read_csv("fields_after.csv")

fields = fields[fields.humus <= 25]

fields.shape

: 

In [ ]:
fields.dtypes

In [ ]:
fields.head()

В них много пропусков (> 70%)

Исследовательский вопрос 1.
Как правильно ухаживать за полем, чтобы повысить плодородие

Как попробуем:
1) Мы хотим сравнить, поэтому поделим поля по уровню плодородия: пусть будут перцентили Ярика
2) Смотрим на ухаживаение -> таблица works.csv + value.csv
3) "Как правильно" -> нужно выделить определенные паттерны для отдельных культур или по климатическим зонам

Конкретно:
1) таргет -> процентная урожайность по площади
2) независимые -> Работы/подработы/машины/площадь работ/глубина обработки/продолжительность работ + ТМЦ (Материалы, используемые в организации для производства продукции или хозяйственных нужд. У нас -- удобрения и тп)
3) Как выделяем паттерны:
    3.1) рассмотрим порядок работ/подработ
    3.2) характеристики самих работ
    3.3) ТМЦ, дозы и периодичность

Возможно что-то упустил

Умные мысли из интернета

1) культуры чередуют
Систематическое чередование однолетних культур позволяет избежать истощения почвы, ежегодно получать хороший урожай без необходимости оставлять часть земли пустой (для «отдыха»)

In [ ]:
# fields['crop'] = fields['crop'].map(lambda x: tuple(sorted(x.split(';'))))
# fields['crop_predecessor'] = fields['crop_predecessor'].map(lambda x: x if x is np.nan else tuple(sorted(x.split(';'))))
# crops = fields[['crop', 'crop_predecessor']]

In [ ]:
# # пусть 
# -2 не допустимо
# -1 не рационально
# 1 допустимо
# 2 оптимально

# crop     crop_predecessor

def crop2set(x):
    if type(x) != str:
        return x
    return list(set(x.split(';')))
fields['crop'] = fields['crop'].map(crop2set)
fields['crop_predecessor'] = fields['crop_predecessor'].map(crop2set)

all_varieties_2024 = []
all_varieties_2023 = []
for el_2024, el_2023 in zip(fields['crop'].values, fields['crop_predecessor'].values):
    all_varieties_2024.extend(el_2024)
    all_varieties_2023.extend(el_2023)
all_varieties_2024 = list(set(all_varieties_2024))
all_varieties_2023 = list(set(all_varieties_2023))
len(all_varieties_2023), len(all_varieties_2024)

In [ ]:
all_varieties_2024

In [ ]:
#works = pd.read_csv("works.csv").drop("Unnamed: 0", axis=1)
works = pd.read_csv("works.csv", encoding="1251", sep=';')
works = works[works['field'].isin(fields['field'])]
works.head(1)

In [ ]:
def norm_yield(x):
    if type(x) != str:
        return x
    if '.' in x:
        a = x.split('.')
    else:
        a = x.split(',')
    return int(a[0]) + (int(a[1]) / 100)
works['yield_fact'] = works['yield_fact'].map(norm_yield)
works['area'] = works['area'].map(norm_yield)
works['area'].describe()

Мерой сравнения будет медиана фактической урожайности поля за оба года

In [ ]:
field2yield = works[['field', 'yield_fact']][works['yield_fact'] > 0].groupby('field').median()['yield_fact'].to_dict()
fields['median_yield'] = fields['field'].map(field2yield)
print(fields['median_yield'].isna().sum())
# заполню тупо по медиане
fields['median_yield'] = fields['median_yield'].fillna(fields['median_yield'].median())
Q = fields['median_yield'].quantile([0.25, 0.75])
IQR = Q.iloc[1] - Q.iloc[0]
fields = fields[(fields['median_yield'] >= Q.iloc[0] - IQR) & (fields['median_yield'] <= Q.iloc[1] + IQR)]
fields['median_yield'].describe()

In [ ]:
works.shape

In [ ]:
# Будет 4 группы, как поделил Ярик. Урожайность плановая * площадь и по перцентилям
yield_mean = fields['median_yield'].mean()
low_fields = fields[fields['median_yield'] <= yield_mean]['field']
high_fields = fields[fields['median_yield'] > yield_mean]['field']
print(low_fields.shape, high_fields.shape)

low_fields_works = works[works['field'].isin(low_fields)]
high_fields_works = works[works['field'].isin(high_fields)]
low_fields_works.shape, high_fields_works.shape

In [ ]:
import matplotlib.pyplot as plt
low_x = low_fields_works['work_type'].value_counts()
high_x = high_fields_works['work_type'].value_counts()

vals = low_x.values
labels = low_x.index
plt.pie(vals, labels=labels, autopct='%1.1f%%')
plt.title("Работы + Низкий урожай")
plt.axis("equal")
plt.show()

vals = high_x.values
labels = high_x.index
plt.pie(vals, labels=labels, autopct='%1.1f%%')
plt.title("Работы + Высокий урожай")
plt.axis("equal")
plt.show()

In [ ]:
low_fields_works.isna().sum()

In [ ]:
high_fields_works.isna().sum()

In [ ]:
low_fields_works = low_fields_works.drop([
    "Глубина обработки (только для работ с типом \"Почвообработка\")",
    'Назначение готовой продукции',
    'Содержание сухого вещества, средневзвешенное с поля',
    'Влажность зерна средневзвешенная с поля',
    'Сорная примесь средневзвешенная с поля'
    ], axis=1)
high_fields_works = high_fields_works.drop([
    'Глубина обработки (только для работ с типом "Почвообработка")',
    'Назначение готовой продукции',
    'Содержание сухого вещества, средневзвешенное с поля',
    'Влажность зерна средневзвешенная с поля',
    'Сорная примесь средневзвешенная с поля'
], axis=1)

In [ ]:
# plan and культура

def coor_cat_num(df, numeric, categ): # функция для подсчета корреляции для категор/номинал
    inGroups = 0
    betweenGroups = 0

    # общее среднее
    df_mean = df[numeric].mean()
    for val in df[categ].unique():
        # для каждого уникального значения берем столбец
        data = df[df[categ] == val][numeric].dropna()
        if len(data) == 0:
            continue
        
        # добавляем это во внутригрупповые
        inGroups += ((data - data.mean())**2).sum()
        # Это в межгрупповые
        betweenGroups += len(data) * (data.mean() - df_mean) ** 2
    # выводим
    return (betweenGroups / (inGroups + betweenGroups)) ** 0.5


coor_cat_num(low_fields_works, "Площадь, выполненной работы", "Тип работы"),\
coor_cat_num(high_fields_works, "Площадь, выполненной работы", "Тип работы")

In [ ]:
low_fields_works[["Площадь, выполненной работы", "Тип работы"]].groupby("Тип работы").median()

In [ ]:
high_fields_works[["Площадь, выполненной работы", "Тип работы"]].groupby("Тип работы").median()

Видимо не на всех полях проводились работы

In [ ]:
import warnings
warnings.filterwarnings("ignore")
areas = fields[['field', 'area']]
x = works[['Номер поля', "Площадь, выполненной работы"]].groupby('Номер поля').median()['Площадь, выполненной работы'].to_dict()
areas['S work'] = areas['field'].map(x.get)
print(areas.isna().sum())
areas = areas.dropna()
areas.shape

In [ ]:
fields['field'].isin(works['Номер поля']).sum() / len(fields)

In [ ]:
# медианная площадь полей в группах
low_area = fields[['field', 'area']][fields['field'].isin(low_fields_works['Номер поля'])]['area'].median()
high_area = fields[['field', 'area']][fields['field'].isin(high_fields_works['Номер поля'])]['area'].median()

print(low_area, high_area)

areas[['area', 'S work']].corr()